Wills Kookogey PyTorch Learning Enviromnment

In [2]:
import fennec_ml as fn
import json
import os

# --- FOLDER CONFIG ---
cwd = os.getcwd()
main_dir = os.path.dirname(cwd)
data_dir = os.path.join(main_dir, "DATA")
os.makedirs(data_dir, exist_ok=True)
excel_dir = os.path.join(data_dir, "RAW_DATA")
os.makedirs(excel_dir, exist_ok=True)
csv_dir = os.path.join(data_dir, "PROCESSED_DATA")
os.makedirs(csv_dir, exist_ok=True)

json_path = os.path.join(main_dir, "vars_of_interest.json")

# --- EXCEL TO CSV ---
print("Using folder_cleaner() to convert from raw excel files to useful .csv's.\nfolder_cleaner() output:")
fn.folder_cleaner(excel_dir, csv_dir, overwrite= True)

# --- SCALING AND LABELS ---
scaled_data = fn.standardize(csv_dir)
# scaled_data = fn.normalize(csv_dir)
labels = fn.get_2D_CG_labels(csv_dir)

# --- SEGMENTING AND SPLITTING ---
print("\nUsing segment_and_split() to cut data into training/validtion/testing sets\nsegment_and_split() output:")
# using default 70%/15%/15% train/val/test split
dataset_dict = fn.segment_and_split(scaled_data, labels, timesteps = 10)


# # set directories using clean_gerald data for test
# data_dir = os.path.join(os.getcwd(), "DATA/RAW_DATA/GERALD_CLEAN")
# os.makedirs(data_dir, exist_ok=True)
# excel_dir = os.path.join(data_dir, "RAW_DATA")
# os.makedirs(excel_dir, exist_ok=True)
# csv_dir = os.path.join(data_dir, "PROCESSED_DATA")
# os.makedirs(csv_dir, exist_ok=True)

# # excel to csv
# print("Using folder_cleaner() to convert from raw excel files to useful .csv's.\nfolder_cleaner() output:")
# fn.folder_cleaner(excel_dir, csv_dir, overwrite= True)

# # scaling and labels
# scaled_data = fn.standardize(csv_dir)
# labels = fn.get_CG_labels(csv_dir)

# # segment and split data with default 70%/15%/15% train/val/test split
# print("\nUsing segment_and_split() to cut data into training/validtion/testing sets\nsegment_and_split() output:")
# dataset_dict = fn.segment_and_split(scaled_data, labels, timesteps = 10)



Using folder_cleaner() to convert from raw excel files to useful .csv's.
folder_cleaner() output:
065G_AAC_L_2.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 065G_AAC_L_2.csv
084G_CCC_L_2.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 084G_CCC_L_2.csv
083G_CCC_L_3.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 083G_CCC_L_3.csv
069G_BBC_S_4.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 069G_BBC_S_4.csv
059G_AAC_L_1.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 059G_AAC_L_1.csv
081G_CCS_L_2.xlsx processed and saved to /Users/willsstoddard/Documents/Development/Python/FENNEC/FENNEC-25_26/DATA/PROCESSED_DATA as 081G_CCS_L_2.